In [1]:
using Plots
using CSV
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
#体を揺すって脚を出す歩容

zc=0.12
g=9.8
dt=0.0025
L=0.16
l=0.03 #歩幅
#x=0.03
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.4/dt)
#both foot support point time
Tdl=Int32(0.0025/dt)
#preview step
M=150
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        
        ] #  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
        
         ]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;3;0]#上げる脚とステップの関係

#初期姿勢
f_zmp_x=abs(real_sx[1]-real_sx[2])/2+0.00
f_zmp_y=L/2+0.01

e_zmp_x=abs(real_sx[1]-real_sx[2])/2+(l/3)
e_zmp_y=L/2+0.0 #追加 +0.01  多分0.0でもok

#Virtual support point
sx=[f_zmp_x
    real_sx[8]+abs(real_sx[4]-real_sx[8])/2+0.005+0.005
    
    e_zmp_x]

sy=[f_zmp_y
    abs(real_sy[7]-real_sy[8])/2+0.005
    
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[20000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t*dt,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t*dt,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t*dt,px,lab="Target ZMP",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip1802)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,55.1626 40.5169,55.1626 41.6637,55.1626 42.8105,55.1626 43.9573,55.1626 45.1041,55.1626 46.2509,55.1626 47.3977,55.1626 48.5445,55.1626 49.6912,55.1626 
 50.838,55.1626 51.9848,55.1626 53.1316,55.1626 54.2784,55.1626 55.4252,55.1626 56.572,55.1626 57.7188,55.1626 58.8656,55.1626 60.0124,55.1626 61.1592,55.1626 
 62.306,55.1626 63.4528,55.1626 64.5996,55.1626 65.7464,55.1626 66.8932,55.1626 68.04,55.1626 69.1868,55.1626 70.3336,55.1626 71.4804,55.1626 72.6272,55.1626 
 73.774,55.1626 74.9208,55.1626 76.0676,55.1626 77.2144,55.1626 78.3612,55.1626 79.508,55.1626 80.6548,55.1626 81.8015,55.1626 82.9483,55.1626 84.0951,55.1626 
 85.2419,55.1626 86.3887,55.1626 87.5355,55.1626 88.6823,55.1626 89.8291,55.1626 90.9759,55.1626 92.1227,55.1626 93.2695,55.1626 94.4163,55.1626 95.5631,55.1626 
 96.7099,55.1626 97.8567,55.1626 99.0035,55.1626 100.15,55.1626 101.297,55.1626 102.444,55.1626 103.591,55.1626 104.737,55.1626 105.884,55.1626 107.031,55.1626 
 108.178,55.1626 109.325,55.1626 110.471,55.1626 111.618,55.1626 112.765,55.1626 113.912,55.1626 115.059,55.1626 116.205,55.1626 117.352,55.1626 118.499,55.1626 
 119.646,55.1626 120.793,55.1626 121.939,55.1626 123.086,55.1626 124.233,55.1626 125.38,55.1626 126.527,55.1626 127.673,55.1626 128.82,55.1626 129.967,55.1626 
 131.114,55.1626 132.261,55.1626 133.407,55.1626 134.554,55.1626 135.701,55.1626 136.848,55.1626 137.995,55.1626 139.141,55.1626 140.288,55.1626 141.435,55.1626 
 142.582,55.1626 143.729,55.1626 144.875,55.1626 146.022,55.1626 147.169,55.1626 148.316,55.1626 149.463,55.1626 150.609,55.1626 151.756,55.1626 152.903,55.1626 
 154.05,55.1626 155.197,55.1626 156.343,55.1626 157.49,55.1626 158.637,55.1626 159.784,55.1626 160.931,55.1626 162.077,55.1626 163.224,55.1626 164.371,55.1626 
 165.518,55.1626 166.664,55.1626 167.811,55.1626 168.958,55.1626 170.105,55.1626 171.252,55.1626 172.398,55.1626 173.545,55.1626 174.692,55.1626 175.839,55.1626 
 176.986,55.1626 178.132,55.1626 179.279,55.1626 180.426,55.1626 181.573,55.1626 182.72,55.1626 183.866,55.1626 185.013,55.1626 186.16,55.1626 187.307,55.1626 
 188.454,55.1626 189.6,55.1626 190.747,55.1626 191.894,55.1626 193.041,55.1626 194.188,55.1626 195.334,55.1626 196.481,55.1626 197.628,55.1626 198.775,55.1626 
 199.922,55.1626 201.068,55.1626 202.215,55.1626 203.362,55.1626 204.509,55.1626 205.656,55.1626 206.802,55.1626 207.949,55.1626 209.096,55.1626 210.243,55.1626 
 211.39,55.1626 212.536,55.1626 213.683,55.1626 214.83,55.1626 215.977,55.1626 217.124,55.1626 218.27,55.1626 219.417,55.1626 220.564,55.1626 221.711,55.1626 
 222.858,9.93569 224.004,9.93569 225.151,9.93569 226.298,9.93569 227.445,9.93569 228.591,9.93569 229.738,9.93569 230.885,9.93569 232.032,9.93569 233.179,9.93569 
 234.325,9.93569 235.472,9.93569 236.619,9.93569 237.766,9.93569 238.913,9.93569 240.059,9.93569 241.206,9.93569 242.353,9.93569 243.5,9.93569 244.647,9.93569 
 245.793,9.93569 246.94,9.93569 248.087,9.93569 249.234,9.93569 250.381,9.93569 251.527,9.93569 252.674,9.93569 253.821,9.93569 254.968,9.93569 256.115,9.93569 
 257.261,9.93569 258.408,9.93569 259.555,9.93569 260.702,9.93569 261.849,9.93569 262.995,9.93569 264.142,9.93569 265.289,9.93569 266.436,9.93569 267.583,9.93569 
 268.729,9.93569 269.876,9.93569 271.023,9.93569 272.17,9.93569 273.317,9.93569 274.463,9.93569 275.61,9.93569 276.757,9.93569 277.904,9.93569 279.051,9.93569 
 280.197,9.93569 281.344,9.93569 282.491,9.93569 283.638,9.93569 284.785,9.93569 285.931,9.93569 287.078,9.93569 288.225,9.93569 289.372,9.93569 290.519,9.93569 
 291.665,9.93569 292.812,9.

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip1902)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,19.3937 40.5169,19.3937 41.6637,19.3937 42.8105,19.3937 43.9573,19.3937 45.1041,19.3937 46.2509,19.3937 47.3977,19.3937 48.5445,19.3937 49.6912,19.3937 
 50.838,19.3937 51.9848,19.3937 53.1316,19.3937 54.2784,19.3937 55.4252,19.3937 56.572,19.3937 57.7188,19.3937 58.8656,19.3937 60.0124,19.3937 61.1592,19.3937 
 62.306,19.3937 63.4528,19.3937 64.5996,19.3937 65.7464,19.3937 66.8932,19.3937 68.04,19.3937 69.1868,19.3937 70.3336,19.3937 71.4804,19.3937 72.6272,19.3937 
 73.774,19.3937 74.9208,19.3937 76.0676,19.3937 77.2144,19.3937 78.3612,19.3937 79.508,19.3937 80.6548,19.3937 81.8015,19.3937 82.9483,19.3937 84.0951,19.3937 
 85.2419,19.3937 86.3887,19.3937 87.5355,19.3937 88.6823,19.3937 89.8291,19.3937 90.9759,19.3937 92.1227,19.3937 93.2695,19.3937 94.4163,19.3937 95.5631,19.3937 
 96.7099,19.3937 97.8567,19.3937 99.0035,19.3937 100.15,19.3937 101.297,19.3937 102.444,19.3937 103.591,19.3937 104.737,19.3937 105.884,19.3937 107.031,19.3937 
 108.178,19.3937 109.325,19.3937 110.471,19.3937 111.618,19.3937 112.765,19.3937 113.912,19.3937 115.059,19.3937 116.205,19.3937 117.352,19.3937 118.499,19.3937 
 119.646,19.3937 120.793,19.3937 121.939,19.3937 123.086,19.3937 124.233,19.3937 125.38,19.3937 126.527,19.3937 127.673,19.3937 128.82,19.3937 129.967,19.3937 
 131.114,19.3937 132.261,19.3937 133.407,19.3937 134.554,19.3937 135.701,19.3937 136.848,19.3937 137.995,19.3937 139.141,19.3937 140.288,19.3937 141.435,19.3937 
 142.582,19.3937 143.729,19.3937 144.875,19.3937 146.022,19.3937 147.169,19.3937 148.316,19.3937 149.463,19.3937 150.609,19.3937 151.756,19.3937 152.903,19.3937 
 154.05,19.3937 155.197,19.3937 156.343,19.3937 157.49,19.3937 158.637,19.3937 159.784,19.3937 160.931,19.3937 162.077,19.3937 163.224,19.3937 164.371,19.3937 
 165.518,19.3937 166.664,19.3937 167.811,19.3937 168.958,19.3937 170.105,19.3937 171.252,19.3937 172.398,19.3937 173.545,19.3937 174.692,19.3937 175.839,19.3937 
 176.986,19.3937 178.132,19.3937 179.279,19.3937 180.426,19.3937 181.573,19.3937 182.72,19.3937 183.866,19.3937 185.013,19.3937 186.16,19.3937 187.307,19.3937 
 188.454,19.3937 189.6,19.3937 190.747,19.3937 191.894,19.3937 193.041,19.3937 194.188,19.3937 195.334,19.3937 196.481,19.3937 197.628,19.3937 198.775,19.3937 
 199.922,19.3937 201.068,19.3937 202.215,19.3937 203.362,19.3937 204.509,19.3937 205.656,19.3937 206.802,19.3937 207.949,19.3937 209.096,19.3937 210.243,19.3937 
 211.39,19.3937 212.536,19.3937 213.683,19.3937 214.83,19.3937 215.977,19.3937 217.124,19.3937 218.27,19.3937 219.417,19.3937 220.564,19.3937 221.711,19.3937 
 222.858,213.344 224.004,213.344 225.151,213.344 226.298,213.344 227.445,213.344 228.591,213.344 229.738,213.344 230.885,213.344 232.032,213.344 233.179,213.344 
 234.325,213.344 235.472,213.344 236.619,213.344 237.766,213.344 238.913,213.344 240.059,213.344 241.206,213.344 242.353,213.344 243.5,213.344 244.647,213.344 
 245.793,213.344 246.94,213.344 248.087,213.344 249.234,213.344 250.381,213.344 251.527,213.344 252.674,213.344 253.821,213.344 254.968,213.344 256.115,213.344 
 257.261,213.344 258.408,213.344 259.555,213.344 260.702,213.344 261.849,213.344 262.995,213.344 264.142,213.344 265.289,213.344 266.436,213.344 267.583,213.344 
 268.729,213.344 269.876,213.344 271.023,213.344 272.17,213.344 273.317,213.344 274.463,213.344 275.61,213.344 276.757,213.344 277.904,213.344 279.051,213.344 
 280.197,213.344 281.344,213.344 282.491,213.344 283.638,213.344 284.785,213.344 285.931,213.344 287.078,213.344 288.225,213.344 289.372,213.344 290.519,213.344 
 291.665,213.344 292.812,213.344 293.959,213.344 295.106,213.344 296.252,213.344 2

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
plot!(px,py,lab="ZoM",linewidth=1,linecolor="black") #追加
plot!(real_sx,real_sy,st=:scatter)
plot!(sx,sy,st=:scatter)
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
<polyline clip-path="url(#clip2002)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 245.163,166.925 245.163,166.925 
 245.163,166.925 245.163,166.925 245.163,166.926 245.163,166.926 245.164,166.926 245.164,166.927 245.164,166.928 245.165,166.93 
 245.165,166.933 245.166,166.936 245.167,166.94 245.169,166.945 245.17,166.952 245.172,166.96 245.175,166.969 245.177,166.98 245.181,166.992 245.184,167.006 
 245.188,167.023 245.193,167.041 245.199,167.062 245.204,167.085 245.211,167.11 245.218,167.138 245.226,167.168 245.235,167.201 245.244,167.237 245.254,167.276 
 245.265,167.317 245.276,167.362 245.288,167.41 245.302,167.461 245.316,167.515 245.33,167.573 245.346,167.633 245.363,167.698 245.38,167.765 245.398,167.836 
 245.418,167.911 245.438,167.989 245.459,168.071 245.481,168.156 245.504,168.246 245.528,168.338 245.553,168.435 245.579,168.536 245.606,168.64 245.634,168.748 
 245.663,168.86 245.693,168.976 245.724,169.096 245.756,169.22 245.789,169.348 245.823,169.48 245.858,169.616 245.894,169.756 245.931,169.901 245.97,170.05 
 246.009,170.203 246.05,170.361 246.092,170.523 246.135,170.689 246.179,170.861 246.225,171.037 246.271,171.217 246.319,171.403 246.368,171.593 246.419,171.788 
 246.47,171.989 246.523,172.194 246.578,172.405 246.634,172.621 246.691,172.843 246.75,173.07 246.81,173.304 246.872,173.543 246.935,173.788 247,174.039 
 247.066,174.297 247.134,174.561 247.204,174.832 247.276,175.109 247.349,175.394 247.425,175.685 247.502,175.984 247.581,176.291 247.662,176.605 247.745,176.927 
 247.83,177.257 247.918,177.595 248.007,177.942 248.099,178.298 248.193,178.662 248.29,179.036 248.389,179.419 248.49,179.812 248.594,180.214 248.7,180.626 
 248.809,181.049 248.921,181.482 249.036,181.925 249.153,182.38 249.273,182.846 249.396,183.322 249.522,183.811 249.651,184.311 249.783,184.823 249.919,185.346 
 250.057,185.883 250.199,186.431 250.344,186.992 250.492,187.566 250.643,188.153 250.798,188.752 250.956,189.365 251.118,189.99 251.282,190.629 251.451,191.282 
 251.623,191.947 251.798,192.626 251.976,193.318 252.158,194.023 252.344,194.741 252.533,195.473 252.725,196.217 252.921,196.975 253.119,197.745 253.321,198.527 
 253.526,199.322 253.735,200.129 253.946,200.948 254.16,201.778 254.377,202.619 254.597,203.47 254.82,204.332 255.045,205.204 255.272,206.085 255.502,206.974 
 255.734,207.872 255.967,208.778 256.203,209.691 256.44,210.61 256.679,211.534 256.919,212.464 257.16,213.398 257.402,214.336 257.645,215.277 257.888,216.219 
 258.132,217.163 258.376,218.108 258.62,219.053 258.864,219.997 259.107,220.94 259.35,221.881 259.592,222.82 259.834,223.756 260.075,224.688 260.314,225.616 
 260.552,226.539 260.789,227.457 261.025,228.368 261.258,229.273 261.49,230.171 261.72,231.061 261.948,231.943 262.173,232.816 262.396,233.679 262.617,234.533 
 262.835,235.376 263.05,236.208 263.263,237.029 263.472,237.838 263.679,238.636 263.883,239.42 264.084,240.192 264.282,240.951 264.476,241.696 264.668,242.427 
 264.856,243.145 265.041,243.848 265.222,244.537 265.401,245.212 265.576,245.871 265.748,246.516 265.916,247.147 266.081,247.762 266.243,248.362 266.401,248.947 
 266.557,249.517 266.709,250.073 266.857,250.613 267.003,251.138 267.145,251.648 267.284,252.143 267.42,252.624 267.553,253.09 267.682,253.541 267.809,253.978 
 267.933,254.4 268.054,254.808 268.172,255.202 268.287,255.583 268.399,255.949 268.508,256.301 268.615,256.641 268.719,256.966 268.82,257.279 268.919,257.579 
 269.016,257.865 269.109,258.139 269.201,258.401 269.29,258.65 269.377,258.887 269.461,259.112 269.543,259.325 269.624,259.526 269.702,259.716 269.777,259.894 
 269.851,260.061 269.923,260.217 269.993,260.362 270.

In [9]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end

function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
   
    A_z=[zh; ze; zf; ve]

    B_z=[th^3 th^2 th 1
        te^3 te^2 te 1
        tf^3 tf^2 tf 1
        3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
     z=C_z[1]*.^(t,3)+C_z[2]*.^(t,2)+C_z[3]*t+C_z[4] #位置
    v=3*C_z[1]*.^(t,2)+2*C_z[2]*t+C_z[3] #速度
    a=6*C_z[1]*t+2*C_z[2]
    return z
end



In [10]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=-0.1 #遊脚が接地する速度

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(i!=1)
            #脚を挙げないようにコメントアウト
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.02,-zc+0.01,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

In [11]:
function generate_csv(dataframe)
    CSV.write("leg_trj.csv",dataframe,delim=',',header=false)
end

generate_csv (generic function with 1 method)

In [12]:
df = DataFrame(xr=sx_from_com[1,:],
                xc=sx_from_com[2,:],
                xl=sx_from_com[3,:],
               yr=sy_from_com[1,:],
                yc=sy_from_com[2,:],
                yl=sy_from_com[3,:],
                zr=sz_from_com[1,:],
                zc=sz_from_com[2,:],
                zl=sz_from_com[3,:])
df=generate_csv(df)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: Base.Dates.DateFormat(Base.Dates.Slot[Base.Dates.DelimitedSlot{Base.Dates.Year}(Base.Dates.Year,'y',4,"-"),Base.Dates.DelimitedSlot{Base.Dates.Month}(Base.Dates.Month,'m',2,"-"),Base.Dates.DelimitedSlot{Base.Dates.Day}(Base.Dates.Day,'d',2,r"(?=\s|$)")],"","english"),IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1),"leg_trj.csv",0,false,String["xr","xc","xl","yr","yc","yl","zr","zc","zl"],false)